In [3]:
import dash
from dash import dcc, html
from dash.dependencies import Input, Output
import plotly.graph_objects as go
import pandas as pd

# Load the dataset
df_data = pd.read_csv('../../data/outputs/combined_filtered_data.csv')

# List the companies in the dataset
companies = df_data['Company'].unique()

available_columns = ['Adj Close', 'Volume', 'Open', 'Close', 'High', 'Low']

In [4]:
# Create the layout for the Dash app without the H1 title
app = dash.Dash(__name__)

app.layout = html.Div([
    # Div to hold both dropdowns in a horizontal layout
    html.Div([
        # Dropdown for selecting companies (multi-select)
        dcc.Dropdown(
            id='company-filter',
            options=[{'label': company, 'value': company} for company in companies],
            value=companies,  # Initially select all companies
            multi=True,
            placeholder="Select companies",
            style={'width': '45%', 'display': 'inline-block'}  # Adjust width
        ),
        
        # Dropdown for selecting the metric
        dcc.Dropdown(
            id='metric-dropdown',
            options=[{'label': metric, 'value': metric} for metric in available_columns],
            value='Adj Close',  # Default to 'Adj Close'
            clearable=False,
            style={'width': '45%', 'display': 'inline-block', 'margin-left': '5%'}  # Adjust width and space
        )
    ], style={'display': 'flex', 'justify-content': 'center'}),  # Ensure the divs are centered
    
    # Graph to display the filtered data
    dcc.Graph(id='data-graph')
])

# Callback to update the graph based on selected companies and metrics
@app.callback(
    Output('data-graph', 'figure'),
    [Input('company-filter', 'value'), Input('metric-dropdown', 'value')]
)
def update_graph(selected_companies, selected_metric):
    # Filter the dataframe based on selected companies
    filtered_df = df_data[df_data['Company'].isin(selected_companies)]
    
    # Create the figure
    fig = go.Figure()

    # Add traces for each selected company
    for company in selected_companies:
        company_data = filtered_df[filtered_df['Company'] == company]
        fig.add_trace(go.Scatter(x=company_data['Date'], y=company_data[selected_metric],
                                 mode='lines', name=company))
    
    # Update the layout without a title
    fig.update_layout(
        xaxis_title="Date",
        yaxis_title=selected_metric,
        showlegend=True  # Keep the legend visible
    )
    
    return fig

# Run the app (disable debug mode to remove the blue circle)
if __name__ == '__main__':
    app.run_server(debug=False, port=8054)